In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [3]:
orderbook = pd.read_csv('/Users/tanvipotdar/Projects/LOBster/data_tqap/INTC_2015-01-01_2015-01-31_10/INTC_2015-01-02_34200000_57600000_orderbook_10.csv')
orderbook['midprice'] = (orderbook.ask_price_1+orderbook.bid_price_1)/2
orderbook.head()
len(orderbook)

887629

In [4]:
x = orderbook.drop('midprice',axis=1)
y = orderbook[['midprice']]
x.head()

,ask_price_1,ask_size_1,bid_price_1,bid_size_1,ask_price_2,ask_size_2,bid_price_2,bid_size_2,ask_price_3,ask_size_3,...,bid_price_8,bid_size_8,ask_price_9,ask_size_9,bid_price_9,bid_size_9,ask_price_10,ask_size_10,bid_price_10,bid_size_10
0,367300,4000,365200,400,367500,200,364800,300,367900,300,...,361800,200,372900,1000,361500,200,373400,2500,360800,200
1,367300,4000,365200,400,367500,200,364800,300,367900,300,...,361800,200,372900,1000,361500,200,373400,2500,360800,200
2,367300,4000,365200,100,367500,200,364800,300,367900,300,...,361800,200,372900,1000,361500,200,373400,2500,360800,200
3,367300,4000,365200,100,367500,200,364800,300,367900,300,...,362900,200,372900,1000,361800,200,373400,2500,361500,200
4,367300,4000,365200,100,367500,200,364800,300,367900,300,...,362900,200,372900,1000,361800,200,373400,2500,361500,200


In [11]:
scaler = MinMaxScaler()
x = scaler.fit_transform(x)
y = scaler.fit_transform(y)
scaler.inverse_transform(y)
x_train = x[:20000].reshape(200,100,40)
y_train = y[:20000][::-100][::-1]
x_test = x[20000:887600].reshape(8676,100,40)
y_test = y[20000:887600][::-100][::-1]
x_train.shape

(200, 100, 40)

In [7]:
# define model
# model = tf.keras.Sequential()
# model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(1,2), strides=(1,2), activation='relu', input_shape=(200,100,40)))
# model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(4,1), activation='relu'))
# model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(4,1), activation='relu'))

# model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(1,2), strides=(1,2), activation='relu'))
# model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(4,1), activation='relu'))
# model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(4,1), activation='relu'))

# model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(1,10), strides=(1,2), activation='relu'))
# model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(4,1), activation='relu'))
# model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(4,1), activation='relu'))
y_train

array([[0.45673077],
       [0.53365385],
       [0.53365385],
       [0.54326923],
       [0.49519231],
       [0.48076923],
       [0.47115385],
       [0.46153846],
       [0.47115385],
       [0.48076923],
       [0.5       ],
       [0.50480769],
       [0.51442308],
       [0.5       ],
       [0.51923077],
       [0.51923077],
       [0.54807692],
       [0.53846154],
       [0.54807692],
       [0.5625    ],
       [0.57692308],
       [0.58653846],
       [0.58653846],
       [0.60096154],
       [0.62019231],
       [0.625     ],
       [0.625     ],
       [0.63942308],
       [0.65384615],
       [0.67788462],
       [0.70192308],
       [0.69230769],
       [0.72115385],
       [0.71634615],
       [0.73076923],
       [0.71153846],
       [0.71153846],
       [0.71153846],
       [0.71153846],
       [0.72115385],
       [0.71634615],
       [0.71634615],
       [0.71634615],
       [0.71153846],
       [0.71634615],
       [0.71634615],
       [0.72115385],
       [0.716

In [143]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv1D(filters=16, kernel_size=2, activation='relu', input_shape=(100, 40)))
model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(1))
model.compile(optimizer='adam', loss='mse')

callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]
# model_m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
history = model.fit(x_train, y_train, epochs=1000, verbose=1)

Epoch 1/1000
200/200 [==============================] - 0s 1ms/sample - loss: 0.0522 - mean_squared_error: 0.0522 - mean_absolute_error: 0.2009
Epoch 2/1000
200/200 [==============================] - 0s 131us/sample - loss: 0.0086 - mean_squared_error: 0.0086 - mean_absolute_error: 0.0779
Epoch 3/1000
200/200 [==============================] - 0s 126us/sample - loss: 0.0022 - mean_squared_error: 0.0022 - mean_absolute_error: 0.0412
Epoch 4/1000
200/200 [==============================] - 0s 124us/sample - loss: 0.0013 - mean_squared_error: 0.0013 - mean_absolute_error: 0.0305
Epoch 5/1000
200/200 [==============================] - 0s 126us/sample - loss: 8.9407e-04 - mean_squared_error: 8.9407e-04 - mean_absolute_error: 0.0228
Epoch 6/1000
200/200 [==============================] - 0s 130us/sample - loss: 6.0672e-04 - mean_squared_error: 6.0672e-04 - mean_absolute_error: 0.0184
Epoch 7/1000
200/200 [==============================] - 0s 133us/sample - loss: 5.0070e-04 - mean_squared_erro

200/200 [==============================] - 0s 130us/sample - loss: 8.4794e-05 - mean_squared_error: 8.4794e-05 - mean_absolute_error: 0.0075
Epoch 107/1000
200/200 [==============================] - 0s 140us/sample - loss: 7.5632e-05 - mean_squared_error: 7.5632e-05 - mean_absolute_error: 0.0070
Epoch 108/1000
200/200 [==============================] - 0s 136us/sample - loss: 7.7730e-05 - mean_squared_error: 7.7730e-05 - mean_absolute_error: 0.0070
Epoch 109/1000
200/200 [==============================] - 0s 132us/sample - loss: 1.0616e-04 - mean_squared_error: 1.0616e-04 - mean_absolute_error: 0.0083
Epoch 110/1000
200/200 [==============================] - 0s 138us/sample - loss: 9.3867e-05 - mean_squared_error: 9.3867e-05 - mean_absolute_error: 0.0077
Epoch 111/1000
200/200 [==============================] - 0s 136us/sample - loss: 6.3309e-05 - mean_squared_error: 6.3309e-05 - mean_absolute_error: 0.0063
Epoch 112/1000
200/200 [==============================] - 0s 129us/sample - los

200/200 [==============================] - 0s 133us/sample - loss: 4.2297e-05 - mean_squared_error: 4.2297e-05 - mean_absolute_error: 0.0052
Epoch 211/1000
200/200 [==============================] - 0s 130us/sample - loss: 8.4830e-05 - mean_squared_error: 8.4830e-05 - mean_absolute_error: 0.0073
Epoch 212/1000
200/200 [==============================] - 0s 129us/sample - loss: 6.9000e-05 - mean_squared_error: 6.9000e-05 - mean_absolute_error: 0.0068
Epoch 213/1000
200/200 [==============================] - 0s 128us/sample - loss: 4.6750e-05 - mean_squared_error: 4.6750e-05 - mean_absolute_error: 0.0056
Epoch 214/1000
200/200 [==============================] - 0s 128us/sample - loss: 6.7927e-05 - mean_squared_error: 6.7927e-05 - mean_absolute_error: 0.0065
Epoch 215/1000
200/200 [==============================] - 0s 133us/sample - loss: 4.9908e-05 - mean_squared_error: 4.9908e-05 - mean_absolute_error: 0.0056
Epoch 216/1000
200/200 [==============================] - 0s 125us/sample - los

200/200 [==============================] - 0s 129us/sample - loss: 5.5195e-05 - mean_squared_error: 5.5195e-05 - mean_absolute_error: 0.0060
Epoch 315/1000
200/200 [==============================] - 0s 131us/sample - loss: 3.5656e-05 - mean_squared_error: 3.5656e-05 - mean_absolute_error: 0.0047
Epoch 316/1000
200/200 [==============================] - 0s 126us/sample - loss: 3.1300e-05 - mean_squared_error: 3.1300e-05 - mean_absolute_error: 0.0044
Epoch 317/1000
200/200 [==============================] - 0s 124us/sample - loss: 2.5247e-05 - mean_squared_error: 2.5247e-05 - mean_absolute_error: 0.0039
Epoch 318/1000
200/200 [==============================] - 0s 121us/sample - loss: 4.1207e-05 - mean_squared_error: 4.1207e-05 - mean_absolute_error: 0.0050
Epoch 319/1000
200/200 [==============================] - 0s 128us/sample - loss: 3.1239e-05 - mean_squared_error: 3.1239e-05 - mean_absolute_error: 0.0045
Epoch 320/1000
200/200 [==============================] - 0s 130us/sample - los

200/200 [==============================] - 0s 127us/sample - loss: 1.9744e-05 - mean_squared_error: 1.9744e-05 - mean_absolute_error: 0.0033
Epoch 419/1000
200/200 [==============================] - 0s 136us/sample - loss: 2.9132e-05 - mean_squared_error: 2.9132e-05 - mean_absolute_error: 0.0043
Epoch 420/1000
200/200 [==============================] - 0s 154us/sample - loss: 4.7995e-05 - mean_squared_error: 4.7995e-05 - mean_absolute_error: 0.0057
Epoch 421/1000
200/200 [==============================] - 0s 122us/sample - loss: 3.4966e-05 - mean_squared_error: 3.4966e-05 - mean_absolute_error: 0.0048
Epoch 422/1000
200/200 [==============================] - 0s 121us/sample - loss: 7.9267e-05 - mean_squared_error: 7.9267e-05 - mean_absolute_error: 0.0076
Epoch 423/1000
200/200 [==============================] - 0s 124us/sample - loss: 9.4493e-05 - mean_squared_error: 9.4493e-05 - mean_absolute_error: 0.0081
Epoch 424/1000
200/200 [==============================] - 0s 145us/sample - los

200/200 [==============================] - 0s 125us/sample - loss: 5.1454e-05 - mean_squared_error: 5.1454e-05 - mean_absolute_error: 0.0058
Epoch 523/1000
200/200 [==============================] - 0s 127us/sample - loss: 3.7094e-05 - mean_squared_error: 3.7094e-05 - mean_absolute_error: 0.0049
Epoch 524/1000
200/200 [==============================] - 0s 123us/sample - loss: 2.8376e-05 - mean_squared_error: 2.8376e-05 - mean_absolute_error: 0.0041
Epoch 525/1000
200/200 [==============================] - 0s 129us/sample - loss: 1.9266e-05 - mean_squared_error: 1.9266e-05 - mean_absolute_error: 0.0033
Epoch 526/1000
200/200 [==============================] - 0s 129us/sample - loss: 2.7187e-05 - mean_squared_error: 2.7187e-05 - mean_absolute_error: 0.0041
Epoch 527/1000
200/200 [==============================] - 0s 131us/sample - loss: 3.1678e-05 - mean_squared_error: 3.1678e-05 - mean_absolute_error: 0.0046
Epoch 528/1000
200/200 [==============================] - 0s 122us/sample - los

200/200 [==============================] - 0s 118us/sample - loss: 2.6854e-05 - mean_squared_error: 2.6854e-05 - mean_absolute_error: 0.0043
Epoch 627/1000
200/200 [==============================] - 0s 129us/sample - loss: 5.1590e-05 - mean_squared_error: 5.1590e-05 - mean_absolute_error: 0.0060
Epoch 628/1000
200/200 [==============================] - 0s 125us/sample - loss: 2.5280e-05 - mean_squared_error: 2.5280e-05 - mean_absolute_error: 0.0041
Epoch 629/1000
200/200 [==============================] - 0s 129us/sample - loss: 1.6727e-05 - mean_squared_error: 1.6727e-05 - mean_absolute_error: 0.0031
Epoch 630/1000
200/200 [==============================] - 0s 131us/sample - loss: 1.6032e-05 - mean_squared_error: 1.6032e-05 - mean_absolute_error: 0.0030
Epoch 631/1000
200/200 [==============================] - 0s 127us/sample - loss: 3.6598e-05 - mean_squared_error: 3.6598e-05 - mean_absolute_error: 0.0049
Epoch 632/1000
200/200 [==============================] - 0s 122us/sample - los

200/200 [==============================] - 0s 127us/sample - loss: 2.0273e-05 - mean_squared_error: 2.0273e-05 - mean_absolute_error: 0.0035
Epoch 731/1000
200/200 [==============================] - 0s 125us/sample - loss: 2.0287e-05 - mean_squared_error: 2.0287e-05 - mean_absolute_error: 0.0036
Epoch 732/1000
200/200 [==============================] - 0s 124us/sample - loss: 2.1437e-05 - mean_squared_error: 2.1437e-05 - mean_absolute_error: 0.0037
Epoch 733/1000
200/200 [==============================] - 0s 128us/sample - loss: 2.2146e-05 - mean_squared_error: 2.2146e-05 - mean_absolute_error: 0.0037
Epoch 734/1000
200/200 [==============================] - 0s 126us/sample - loss: 2.1898e-05 - mean_squared_error: 2.1898e-05 - mean_absolute_error: 0.0037
Epoch 735/1000
200/200 [==============================] - 0s 125us/sample - loss: 1.9441e-05 - mean_squared_error: 1.9441e-05 - mean_absolute_error: 0.0035
Epoch 736/1000
200/200 [==============================] - 0s 122us/sample - los

200/200 [==============================] - 0s 131us/sample - loss: 3.5618e-05 - mean_squared_error: 3.5618e-05 - mean_absolute_error: 0.0050
Epoch 835/1000
200/200 [==============================] - 0s 133us/sample - loss: 3.1905e-05 - mean_squared_error: 3.1905e-05 - mean_absolute_error: 0.0048
Epoch 836/1000
200/200 [==============================] - 0s 129us/sample - loss: 1.7232e-05 - mean_squared_error: 1.7232e-05 - mean_absolute_error: 0.0032
Epoch 837/1000
200/200 [==============================] - 0s 128us/sample - loss: 4.1193e-05 - mean_squared_error: 4.1193e-05 - mean_absolute_error: 0.0054
Epoch 838/1000
200/200 [==============================] - 0s 125us/sample - loss: 2.9086e-05 - mean_squared_error: 2.9086e-05 - mean_absolute_error: 0.0045
Epoch 839/1000
200/200 [==============================] - 0s 135us/sample - loss: 3.4060e-05 - mean_squared_error: 3.4060e-05 - mean_absolute_error: 0.0050
Epoch 840/1000
200/200 [==============================] - 0s 128us/sample - los

200/200 [==============================] - 0s 130us/sample - loss: 1.2319e-05 - mean_squared_error: 1.2319e-05 - mean_absolute_error: 0.0028
Epoch 939/1000
200/200 [==============================] - 0s 129us/sample - loss: 1.8932e-05 - mean_squared_error: 1.8932e-05 - mean_absolute_error: 0.0036
Epoch 940/1000
200/200 [==============================] - 0s 127us/sample - loss: 1.3841e-05 - mean_squared_error: 1.3841e-05 - mean_absolute_error: 0.0030
Epoch 941/1000
200/200 [==============================] - 0s 129us/sample - loss: 2.7898e-05 - mean_squared_error: 2.7898e-05 - mean_absolute_error: 0.0043
Epoch 942/1000
200/200 [==============================] - 0s 127us/sample - loss: 1.6701e-05 - mean_squared_error: 1.6701e-05 - mean_absolute_error: 0.0033
Epoch 943/1000
200/200 [==============================] - 0s 129us/sample - loss: 4.1504e-05 - mean_squared_error: 4.1504e-05 - mean_absolute_error: 0.0054
Epoch 944/1000
200/200 [==============================] - 0s 124us/sample - los

In [144]:
y_pred = model.predict(x_test)

In [146]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mean_squared_error(y_pred, y_test)

0.0025966749174864715

In [145]:
scaler.inverse_transform(y_pred)

array([[0.7950666 ],
       [0.81494945],
       [0.8271075 ],
       ...,
       [0.681634  ],
       [0.71826273],
       [0.83544   ]], dtype=float32)

In [1]:
x_train

NameError: name 'x_train' is not defined